In [55]:
# Importing Necessary Libraries
import pandas as pd, numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick

from pathlib import Path

In [ ]:
# Reading data file
DATA = Path("../data/global_superstore_2016.xlsx")
df = pd.read_excel(DATA)

# normalise column names
df.columns = [clm.strip().replace(" ", "_").lower() for clm in df.columns]

# Ensuring Date Time columns are in proper format
for clm in ["order_date", "ship_date"]:
    df[clm] = pd.to_datetime(df[clm], errors="coerce")
    df = df.dropna(subset=[clm]).copy()

# Ensuring the columns are numeric
for clm in ["sales", "quantity", "discount","profit", "shipping_cost" ]:
    if clm in df.columns:
        df[clm] = pd.to_numeric(df[clm], errors="coerce")

# Showing first five records
df.head()

In [ ]:
# df.info() # Summary of DataFrame information

# Quick Exploratory Data Analysis and Quality Checks
date_col = "order_date"
# Checking for duplicate rows
dupes = df.duplicated().sum()
print("Exact duplicate rows:", dupes)

# Setting up directory for reports
FIG = Path("../reports/figures"); FIG.mkdir(parents=True, exist_ok=True)


Exact duplicate rows: 0


In [45]:
# Daily series
daily = df.groupby(df['order_date'].dt.to_period('D')).agg(
    quantity=('quantity','sum'),
    sales=('sales','sum')
).reset_index()
daily['order_date'] = daily['order_date'].dt.to_timestamp()

plt.figure(); plt.plot(daily['order_date'], daily['quantity'])
plt.title("Daily Quantity (Units)"); plt.tight_layout()
plt.savefig(FIG/'daily_quantity.png', dpi=150); plt.close()

plt.figure(); plt.plot(daily['order_date'], daily['sales'])
plt.title("Daily Sales (Revenue)"); plt.tight_layout()
plt.savefig(FIG/'daily_sales.png', dpi=150); plt.close()


In [59]:
# Category / Region summaries
if 'category' in df.columns:
    cat = df.groupby('category').agg(quantity=('quantity','sum')).reset_index().sort_values('quantity', ascending=False)
    plt.figure(); plt.bar(cat['category'], cat['quantity']); plt.title("Quantity by Category"); plt.tight_layout()
    plt.savefig(FIG/'qty_by_category.png', dpi=150); plt.close()

if 'region' in df.columns:
    reg = df.groupby('region').agg(sales=('sales','sum')).reset_index().sort_values('sales', ascending=False)
    plt.figure(); plt.bar(reg['region'], reg['sales']); plt.xticks(rotation="vertical"); plt.title("Sales by Region")
    plt.subplots_adjust(bottom=0.3);  plt.gca().yaxis.set_major_formatter(mtick.StrMethodFormatter('{x:,.0f}'))
    plt.tight_layout()
    plt.savefig(FIG/'sales_by_region.png', dpi=150); plt.close()

In [ ]:
# Setting up directory for Star Schema (BI & modelling)
OUT = Path("../data/star_schema"); OUT.mkdir(parents=True, exist_ok=True)

# Dimension - Date
date_df = pd.DataFrame({"date": pd.date_range(df['order_date'].min(), df['order_date'].max(), freq='D')})
date_df['date_key'] = date_df['date'].dt.strftime('%Y%m%d').astype(int)
date_df['year'] = date_df['date'].dt.year
date_df['quarter'] = date_df['date'].dt.quarter
date_df['month'] = date_df['date'].dt.month
date_df['month_name'] = date_df['date'].dt.strftime('%b')
date_df['week'] = date_df['date'].dt.isocalendar().week.astype(int)
date_df['dow'] = date_df['date'].dt.weekday + 1
date_df['is_weekend'] = date_df['dow'].isin([6,7]).astype(int)
date_df.to_csv(OUT/'dim_date.csv', index=False)

# Dimension - Customer
dim_customer = df[['customer_id','customer_name','segment']].drop_duplicates()
dim_customer.to_csv(OUT/'dim_customer.csv', index=False)

# Dimension - Product
dim_product = df[['product_id','product_name','category','sub-category']].drop_duplicates()
dim_product.to_csv(OUT/'dim_product.csv', index=False)

# Dimension - Geography
geo_cols = [clm for clm in ['country','region','state','city','postal_code'] if clm in df.columns]
dim_geo = df[geo_cols].drop_duplicates().copy()
dim_geo['geo_id'] = range(1, len(dim_geo)+1)
dim_geo.to_csv(OUT/'dim_geo.csv', index=False)

# Dimension - Fact Sales
txn_col = 'order_id' if 'order_id' in df.columns else 'txn_id'
if txn_col not in df.columns: 
    df[txn_col] = range(1, len(df)+1)

fact_cols = [clm for clm in [txn_col, 'order_date','sales','quantity','profit','discount','product_id','customer_id','region','state','city'] if clm in df.columns]
fact = df[fact_cols].copy()
fact['date_key'] = fact['order_date'].dt.strftime('%Y%m%d').astype(int)
fact = fact.merge(dim_geo, on=[clm for clm in ['region','state','city'] if clm in fact.columns], how='left')
fact.to_csv(OUT/'fact_sales.csv', index=False)